TIỀN XỬ LÝ DỮ LIỆU

In [2]:
!pip install emojis

In [26]:
import pandas as pd
import numpy as np
import enchant
import unicodedata
import process as Processor
import utils as Utils

In [8]:
df = pd.read_csv('data.csv')
df.head()

,comment,rating
0,Màu sắc:purple\nĐúng với mô tả:yes\nChất liệu:...,5
1,Màu sắc:xanh dương\nChất liệu:vải\nĐúng với mô...,5
2,Đúng với mô tả:giong mo ta\nChất liệu:thun\nMà...,5
3,Đúng với mô tả:đúng mô tả\nMàu sắc:ghi\nChất l...,5
4,Chất liệu:thun mát\nĐúng với mô tả:đúng\nMàu s...,5


In [3]:
df.shape

(221276, 2)

Tập dữ liệu có 221276 records và 2 features (comment, rating) với:
- comment: đánh giá sản phẩm
- rating: số sao

Đếm tần số xuất hiện của từng rating (sao)

In [4]:
df['rating'].value_counts()

rating
5         178566
3          13004
4          12481
1          10881
2           6059
rating       284
Name: count, dtype: int64

Nhận xét: Tập dữ liệu có hơn 200000 records nhưng có sự chênh lệch lớn giữa đánh giá 5 sao và đánh giá có số sao còn lại. Điều này là hoàn toàn bình thường vì thuật toán gợi ý sản phẩm của Shopee sẽ ưu tiên hiển thị những sản phẩm có đánh giá tốt và hạn chế (hoặc không gợi ý) các sản phẩm có đánh giá kém. Việc này thực sự không thể tránh khỏi.

2.1. Dữ liệu rỗng

In [22]:
df.isnull().sum()

comment    99675
rating         1
dtype: int64

In [9]:
df = df.dropna()
df.shape

(121600, 2)

2.2. Tiến hành đưa tất cả các text về cùng một chuẩn duy nhất:
- Đưa về chữ thường.
- Đưa về cùng chuẩn Unicode (NDF).

- Ta sẽ tạo thêm 1 feature: nomalized_comment được chuẩn hóa 2 bước như trên
- Và feature comment chuẩn hóa nhưng không đưa về chữ thường.

In [10]:
df['nomalized_comment'] = df['comment'].astype(str).apply(lambda cmt: Processor.normalizeComment(cmt, True))
df.head()

,comment,rating,nomalized_comment
0,Màu sắc:purple\nĐúng với mô tả:yes\nChất liệu:...,5,màu sắc:purple\nđúng với mô tả:yes\nch...
1,Màu sắc:xanh dương\nChất liệu:vải\nĐúng với mô...,5,màu sắc:xanh dương\nchất liệu:vải\nđ...
2,Đúng với mô tả:giong mo ta\nChất liệu:thun\nMà...,5,đúng với mô tả:giong mo ta\nchất liệu...
3,Đúng với mô tả:đúng mô tả\nMàu sắc:ghi\nChất l...,5,đúng với mô tả:đúng mô tả\nmàu sắc:...
4,Chất liệu:thun mát\nĐúng với mô tả:đúng\nMàu s...,5,chất liệu:thun mát\nđúng với mô tả:đ...


In [11]:
df['comment'] = df['comment'].astype(str).apply(lambda cmt: Processor.normalizeComment(cmt, False))
df.head()

,comment,rating,nomalized_comment
0,Màu sắc:purple\nĐúng với mô tả:yes\nCh...,5,màu sắc:purple\nđúng với mô tả:yes\nch...
1,Màu sắc:xanh dương\nChất liệu:vải\nĐ...,5,màu sắc:xanh dương\nchất liệu:vải\nđ...
2,Đúng với mô tả:giong mo ta\nChất liệu...,5,đúng với mô tả:giong mo ta\nchất liệu...
3,Đúng với mô tả:đúng mô tả\nMàu sắc:...,5,đúng với mô tả:đúng mô tả\nmàu sắc:...
4,Chất liệu:thun mát\nĐúng với mô tả:đ...,5,chất liệu:thun mát\nđúng với mô tả:đ...


2.3. Đánh giá "quảng cáo":
- Đánh giá chứa link url: Theo quan sát, các đánh giá của các sản phẩm đôi khi sẽ chứa các url do những người bán hàng chèn vào để "quảng cáo" mặt hàng khác của họ. Đây là các noise sample mà ta cần phải loại bỏ khỏi tập dữ liệu.
- Đánh giá mà chữ IN HOA chiểm hơn 50% độ dài đánh giá: Các đánh giá khả năng cao cũng là quảng cáo vì để làm nổi bật đánh giá này so với những đánh giá còn lại.

2.3.1. Đánh giá chứa link url:

In [12]:
#Kiểm tra số lượng đánh giá chứa link url
df['contain_url'] = df['nomalized_comment'].apply(lambda cmt: Processor.containsURL(cmt))
Processor.printAfterProcess(df, 'contain_url')
df.head()

Shape: (121600, 4)
contain_url
0    120887
1       713
Name: count, dtype: int64


,comment,rating,nomalized_comment,contain_url
0,Màu sắc:purple\nĐúng với mô tả:yes\nCh...,5,màu sắc:purple\nđúng với mô tả:yes\nch...,0
1,Màu sắc:xanh dương\nChất liệu:vải\nĐ...,5,màu sắc:xanh dương\nchất liệu:vải\nđ...,0
2,Đúng với mô tả:giong mo ta\nChất liệu...,5,đúng với mô tả:giong mo ta\nchất liệu...,0
3,Đúng với mô tả:đúng mô tả\nMàu sắc:...,5,đúng với mô tả:đúng mô tả\nmàu sắc:...,0
4,Chất liệu:thun mát\nĐúng với mô tả:đ...,5,chất liệu:thun mát\nđúng với mô tả:đ...,0


In [13]:
# Lấy các bình luận không chứa link URL
df = df[df['contain_url'] == 0]
df = df.drop(columns=['contain_url']).reset_index(drop=True) # xóa cột 'contain_url'
Processor.printAfterProcess(df, 'rating')
df.head()

Shape: (120887, 3)
rating
5         95820
1          7467
3          7196
4          6220
2          3900
rating      284
Name: count, dtype: int64


,comment,rating,nomalized_comment
0,Màu sắc:purple\nĐúng với mô tả:yes\nCh...,5,màu sắc:purple\nđúng với mô tả:yes\nch...
1,Màu sắc:xanh dương\nChất liệu:vải\nĐ...,5,màu sắc:xanh dương\nchất liệu:vải\nđ...
2,Đúng với mô tả:giong mo ta\nChất liệu...,5,đúng với mô tả:giong mo ta\nchất liệu...
3,Đúng với mô tả:đúng mô tả\nMàu sắc:...,5,đúng với mô tả:đúng mô tả\nmàu sắc:...
4,Chất liệu:thun mát\nĐúng với mô tả:đ...,5,chất liệu:thun mát\nđúng với mô tả:đ...


2.3.2. Đánh giá IN HOA trên 50%:

In [14]:
#Kiểm tra số lượng đánh giá IN HOA trên 50%
df['contain_adv'] = df['comment'].apply(lambda cmt: Processor.containAdvertisement(cmt))
Processor.printAfterProcess(df, 'contain_adv')
df.head()

Shape: (120887, 4)
contain_adv
False    118668
True       2219
Name: count, dtype: int64


,comment,rating,nomalized_comment,contain_adv
0,Màu sắc:purple\nĐúng với mô tả:yes\nCh...,5,màu sắc:purple\nđúng với mô tả:yes\nch...,False
1,Màu sắc:xanh dương\nChất liệu:vải\nĐ...,5,màu sắc:xanh dương\nchất liệu:vải\nđ...,False
2,Đúng với mô tả:giong mo ta\nChất liệu...,5,đúng với mô tả:giong mo ta\nchất liệu...,False
3,Đúng với mô tả:đúng mô tả\nMàu sắc:...,5,đúng với mô tả:đúng mô tả\nmàu sắc:...,False
4,Chất liệu:thun mát\nĐúng với mô tả:đ...,5,chất liệu:thun mát\nđúng với mô tả:đ...,False


In [15]:
#Loại bỏ đánh giá quảng cáo
df = df[df['contain_adv'] == 0]
df = df.drop(columns=['contain_adv']).reset_index(drop=True) # xóa cột 'contain_adv'
Processor.printAfterProcess(df, 'rating')
df.head()

Shape: (118668, 3)
rating
5         93898
1          7378
3          7137
4          6102
2          3869
rating      284
Name: count, dtype: int64


,comment,rating,nomalized_comment
0,Màu sắc:purple\nĐúng với mô tả:yes\nCh...,5,màu sắc:purple\nđúng với mô tả:yes\nch...
1,Màu sắc:xanh dương\nChất liệu:vải\nĐ...,5,màu sắc:xanh dương\nchất liệu:vải\nđ...
2,Đúng với mô tả:giong mo ta\nChất liệu...,5,đúng với mô tả:giong mo ta\nchất liệu...
3,Đúng với mô tả:đúng mô tả\nMàu sắc:...,5,đúng với mô tả:đúng mô tả\nmàu sắc:...
4,Chất liệu:thun mát\nĐúng với mô tả:đ...,5,chất liệu:thun mát\nđúng với mô tả:đ...


2.4. Kí tự đặc biệt:
2.4.1. Emoji:
- Theo quan sát, ta có thể thấy có các đánh giá chứa emoji. Và các emoji cũng thể hiện một phần thể hiện được sự tích cực hoặc tiêu cực của đánh giá.
- Ta sẽ tiến hành tách các emoji này thành một feature mới trước khi loại bỏ các kí tự đặc biệt vì vốn dĩ emoji cũng được xây dựng bằng các kí tự đặc biệt.
2.4.2. Xóa các kí tự đặc biệt

In [18]:
#Tách emoji sang feature mới:
df['emoji'] = df['comment'].apply(lambda cmt: Processor.extractEmoji(cmt))
df.head()

,comment,rating,nomalized_comment,emoji
0,Màu sắc:purple\nĐúng với mô tả:yes\nCh...,5,màu sắc:purple\nđúng với mô tả:yes\nch...,
1,Màu sắc:xanh dương\nChất liệu:vải\nĐ...,5,màu sắc:xanh dương\nchất liệu:vải\nđ...,
2,Đúng với mô tả:giong mo ta\nChất liệu...,5,đúng với mô tả:giong mo ta\nchất liệu...,
3,Đúng với mô tả:đúng mô tả\nMàu sắc:...,5,đúng với mô tả:đúng mô tả\nmàu sắc:...,
4,Chất liệu:thun mát\nĐúng với mô tả:đ...,5,chất liệu:thun mát\nđúng với mô tả:đ...,


In [19]:
df.tail()

,comment,rating,nomalized_comment,emoji
118663,Vua re vua đep,3,vua re vua đep,
118664,Hơi to sop ạ😢,3,hơi to sop ạ😢,😢
118665,Muốn hoàn tiền lại,1,muốn hoàn tiền lại,
118666,Áo ngắn,2,áo ngắn,
118667,Chất liệu:mỏng,1,chất liệu:mỏng,


In [20]:
",".join(df['emoji'])[:200]

',,,,,,,,,😘,,📌,🍀 📢 💗 🎤 😪 📜 👩\u200d🎨 💙,,❤️,,,,,,😆,,,🆕 ❗,,,,,,,,,🌟 👍,,,,,,,,,,,,,,,,,🥰 👉,,,,,,,,,,,,💪 ‼️ ❌,,,,👍,,,,,,,,,,,😅 💰,,,,,,,,,,,,,,,,,1️⃣ 📍 ✅ ♨️ 📌 🌊 ⚡ 2️⃣ 👉,,,,,,,,,,,,‼️ ❤️,,,,,,,,,,,⭐,,,,,,,,,,,,,,👍'

In [22]:
#Xóa các kí tự đặc biệt
df['nomalized_comment'] = df['nomalized_comment'].apply(lambda cmt: Processor.removeSpecialLetters(cmt))
df.tail()

,comment,rating,nomalized_comment,emoji
118663,Vua re vua đep,3,vua re vua đep,
118664,Hơi to sop ạ😢,3,hơi to sop ạ,😢
118665,Muốn hoàn tiền lại,1,muốn hoàn tiền lại,
118666,Áo ngắn,2,áo ngắn,
118667,Chất liệu:mỏng,1,chất liệu mỏng,


2.5. Đánh giá có chữ cái bị lặp:
- Ta cần chuẩn hóa các từ xinhhhhh quá, đẹppppp xỉuuuuu,... thành xinh quá, đẹp xỉu -> loại bỏ những chữ cái bị lặp.
- Tuy nhiên ta cần lưu ý những từ tiếng anh thông dụng được sử dụng trong đánh giá như feedback, shipper. Nếu chỉ xóa chữ cái lặp thông thường sẽ vô tình biến các từ tiếng anh sai nghĩa -> Cần kiểm tra xem từ lặp chữu cái có phải từ tiếng anh không.
=> Ta sẽ xử lý chung với bước 2.6.

2.6. Đánh giá vô nghĩa:
- Ta sẽ tiến hành xóa các đánh giá không phải là tiếng Việt cũng như các đánh giá soạn "đại" để đủ kí tự (theo yêu cầu của Shopee để nhận xu).
- Sử dụng file vocabulary.txt chứa hơn 17000 từ đơn phổ biến của tiếng Việt.
- Ta cần giữ lại vài từ tiếng anh như shipper, sử dụng package enchant để kiểm tra một từ có phải tiếng Anh không. 

In [31]:
#Xây dựng từ điển
file = open('vocabulary.txt', 'r', encoding="utf8")
vocab_lst = file.read().split('\n')
vietnamese_vocab= {}
for line in vocab_lst:
    key = unicodedata.normalize('NFD', line.strip())
    vietnamese_vocab[key] = True
file.close()

english_vocab = enchant.Dict('en_US')

In [34]:
df['nomalized_comment'] = df['nomalized_comment'].apply(lambda cmt: Processor.removeNoiseWord(cmt, vietnamese_vocab, english_vocab))
df.head()

,comment,rating,nomalized_comment,emoji
0,Màu sắc:purple\nĐúng với mô tả:yes\nCh...,5,màu sắc purple đúng với mô tả yes châ...,
1,Màu sắc:xanh dương\nChất liệu:vải\nĐ...,5,màu sắc xanh dương chất liệu vải đú...,
2,Đúng với mô tả:giong mo ta\nChất liệu...,5,đúng với mô tả giong mo ta chất liệu ...,
3,Đúng với mô tả:đúng mô tả\nMàu sắc:...,5,đúng với mô tả đúng mô tả màu sắc g...,
4,Chất liệu:thun mát\nĐúng với mô tả:đ...,5,chất liệu thun mát đúng với mô tả đu...,


In [43]:
df.loc[118457:118458]

,comment,rating,nomalized_comment,emoji
118457,Áo khoác đẹp rẻ rất nên mua,5,áo khoác đẹp rẻ rất nên mua,
118458,Đúng với mô tả:nânnanananananananann\nCh...,5,đúng với mô tả chất liệu quá tốt m...,


In [45]:
df.loc[118389:118390]

,comment,rating,nomalized_comment,emoji
118389,nó đẹpppppppp,5,nó đẹp,
118390,Màu sắc:xấu,1,màu sắc xấu,
